### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
33,42334638,116340,79085,51530,0,0,1129,5,2025-02-27 22:00,2025-02-27 22:00,2,0,4803,5,1
34,42335392,51582,79085,3,0,0,1129,6,2025-02-28 19:33,2025-02-28 19:34,2,0,6593,4,1
35,42335651,79085,77583,51530,0,0,1130,0,2025-03-01 22:00,2025-03-01 22:00,2,0,1521,2,1
36,42084861,124139,79085,1349,21,69,1130,1,2025-03-02 21:59,2025-03-02 21:59,2,1,11920,2,1
37,42339172,79085,28394,51530,0,0,1130,2,2025-03-03 22:00,2025-03-03 22:00,2,2,1395,3,1
38,42339485,79085,86975,51530,0,0,1130,3,2025-03-04 22:00,2025-03-04 22:00,0,1,2683,5,1
39,42084851,79085,79866,1349,22,69,1130,4,2025-03-05 21:59,2025-03-05 21:59,0,3,10545,5,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,116340,42095,30848,1,11,5,1,0,5 raton plus,11,11,8,42334638
1,79085,28288,39535,0,5,4,0,0,5-4-1a,10,9,11,42334638
0,51582,33208,37244,0,16,0,0,0,nowa,10,9,6,42335392
1,79085,36774,32738,0,6,0,0,0,5-4-1a,6,7,7,42335392
0,79085,38180,34222,0,12,1,0,0,4-2-3-1a,10,9,11,42335651
1,77583,31838,35796,0,9,4,0,0,max2,10,10,9,42335651
0,124139,32943,32939,2,7,1,0,0,1-4-6-0,9,8,9,42084861
1,79085,37137,37141,1,6,2,1,0,4-2-3-1a,9,9,11,42084861
0,79085,37626,36858,0,3,0,0,0,5-4-1a,9,9,11,42339172
1,28394,32426,33194,2,7,0,0,0,352new,10,11,10,42339172


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1602

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,47841,22,61,20,1,1,117,19,61598117020010,1349
1,6295,22,48,15,3,4,107,41,48566107015009,1349
2,23755,22,47,14,5,3,103,30,47573103014011,1349
3,79866,22,47,15,2,5,61,23,47538061015005,1349
4,78671,22,38,12,2,8,69,30,38539069012007,1349
5,89569,22,35,11,2,9,57,50,35507057011002,1349
6,124139,22,34,11,1,10,61,57,34504061011008,1349
7,122169,22,30,9,3,10,100,76,30524100009004,1349
8,79085,22,26,7,5,10,44,35,26509044007006,1349
9,48541,22,12,4,0,18,27,135,12392027004001,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()